In [1]:
import os
import sys
import boto3
from time import time
from time import sleep
import rasterio
from rasterio.plot import show
import xarray as xr
import rioxarray

In [2]:
#!pip list

In [3]:
#! free -h

In [4]:
#!pip install rioxarray --user

In [5]:
def _xr_open_rasterio_retry(s3_file_name):
    cnt=20
    sleeptime=6
    while(cnt>0):
        try:
            da = xr.open_rasterio(s3_file_name)
            print('SUCCESS _xr_open_rasterio_retry', s3_file_name, flush=True)
            return da
        except rasterio.errors.RasterioIOError:
                        print("Unexpected error:", sys.exc_info()[0])
                        print('oops',cnt)
                        print('oops',s3_file_name, flush=True)
                        cnt = cnt - 1
                        sleep(sleeptime)

In [6]:
def xr_build_mosaic_ds(bucket, product, tifs):

    start = time()
    my_da_list =[]
    for tif in tifs:
        try:
            da = _xr_open_rasterio_retry(f's3://{bucket}/'+tif)
        except:
            print('error on ', tif, flush=True)
            print('FATAL error on ', tif, flush=True)

        try:
            da = da.squeeze().drop(labels='band')
            da.name=product
            my_da_list.append(da)
            tnow = time()
            elapsed = tnow - start
            print(tif, elapsed)
        except:
            print('FATAL SQUEEZE error on ', tif, flush=True)

    try:
        DS = xr.merge(my_da_list)
        return(DS)
    except:
        print('FATAL MERGE error on ', tif, flush=True)

In [7]:
def s3_push_delete_local(local_file, bucket, bucket_filepath):
        out_bucket = return_my_bucket(bucket_filepath)
        a = bucket_filepath.split('/')
        bucket_filepath = '/'.join(a[1:]) # strip bucket from path
        print('PUSH', out_bucket, bucket_filepath)
        s3 = boto3.client('s3')
        with open(local_file, "rb") as f:
            s3.upload_fileobj(f, out_bucket, bucket_filepath)
        os.remove(local_file)

In [8]:
def _run_command(cmd, verbose=False):
    if verbose:
        print(cmd)
    result = os.system(cmd)
    if result != 0:
        raise Exception('command "%s" failed with code %d.' % (cmd, result))

In [9]:
def cog_create_from_tif(src_tif,dst_cog):
    command = f'rio cogeo create {src_tif} {dst_cog}'
    _run_command(command)

In [10]:
def return_my_bucket(prefix_with_slash):
    a = prefix_with_slash.split('/')
    print('a=',a)
    THE_BUCKET=a[0]
    print("the BUCKET=",THE_BUCKET)
    return THE_BUCKET

In [11]:
def xr_write_geotiff_from_ds(DS, primary_name, out_prefix_path):
    
    print(DS) # DS is the xarray
    print(primary_name) # first tif 
    print(out_prefix_path)

    a = primary_name.split('/')
    just_tif = a[-2] + '/' + a[-1]
    local_tif = a[-1]
    local_cog = 'COG_' + local_tif

    output = out_prefix_path + just_tif
    bucket = 'ws-enduser'
    print(f'OUTPUT=={output}')
    DS.rio.to_raster(local_tif)
    cog_create_from_tif(local_tif, local_cog)
    s3_push_delete_local(local_cog, bucket, output)
    os.remove(local_tif)
    local_xml = local_cog + '.aux.xml'
    os.remove(local_xml)

In [12]:
#12-2014

month = '12'
year = '2008'
bucket = 'ws-enduser'
product = 'etasw'
#product = 'netet'
tifs = ['USA/r37.0_tile0/'+year+'/'+product+'_'+year+month+'.tif',
        'USA/r37.0_tile1/'+year+'/'+product+'_'+year+month+'.tif',
        'USA/r37.0_tile2/'+year+'/'+product+'_'+year+month+'.tif',
        'USA/r37.0_tile3/'+year+'/'+product+'_'+year+month+'.tif',
        'USA/r50.0_tile5/'+year+'/'+product+'_'+year+month+'.tif',
        'USA/r50.0_tile6/'+year+'/'+product+'_'+year+month+'.tif',
        'USA/r50.0_tile7/'+year+'/'+product+'_'+year+month+'.tif',
        'USA/r50.0_tile8/'+year+'/'+product+'_'+year+month+'.tif',
        'USA/r50.0_tile9/'+year+'/'+product+'_'+year+month+'.tif']

In [13]:
# s3_file_name = 's3://ws-enduser/USA/r37.0_tile0/2000/etasw_200001.tif'
# da = xr.open_rasterio(s3_file_name)
# da

In [14]:
DS = xr_build_mosaic_ds(bucket, product, tifs)

SUCCESS _xr_open_rasterio_retry s3://ws-enduser/USA/r37.0_tile0/2008/etasw_200812.tif
USA/r37.0_tile0/2008/etasw_200812.tif 1.451575517654419
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/USA/r37.0_tile1/2008/etasw_200812.tif
USA/r37.0_tile1/2008/etasw_200812.tif 2.652876377105713
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/USA/r37.0_tile2/2008/etasw_200812.tif
USA/r37.0_tile2/2008/etasw_200812.tif 3.793252944946289
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/USA/r37.0_tile3/2008/etasw_200812.tif
USA/r37.0_tile3/2008/etasw_200812.tif 4.945919990539551
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/USA/r50.0_tile5/2008/etasw_200812.tif
USA/r50.0_tile5/2008/etasw_200812.tif 6.191590785980225
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/USA/r50.0_tile6/2008/etasw_200812.tif
USA/r50.0_tile6/2008/etasw_200812.tif 7.369672536849976
SUCCESS _xr_open_rasterio_retry s3://ws-enduser/USA/r50.0_tile7/2008/etasw_200812.tif
USA/r50.0_tile7/2008/etasw_200812.tif 8.798778772354126
SUCCES

In [15]:
DS

<xarray.Dataset>
Dimensions:  (x: 31248, y: 12976)
Coordinates:
  * x        (x) float64 -125.0 -125.0 -125.0 -125.0 ... -60.0 -60.0 -60.0
  * y        (y) float64 23.0 23.0 23.0 23.01 23.01 ... 49.99 49.99 50.0 50.0
Data variables:
    etasw    (y, x) float64 -7.846e+39 -7.846e+39 ... -7.846e+39 -7.846e+39
Attributes:
    transform:           (0.0020803384719955188, 0.0, -125.00034453048596, 0....
    crs:                 +init=epsg:4326
    res:                 (0.0020803384719955188, -0.0020810045000000006)
    is_tiled:            1
    nodatavals:          (nan,)
    scales:              (1.0,)
    offsets:             (0.0,)
    descriptions:        ('etasw_',)
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  NEAREST

In [16]:
# DS = xr_build_mosaic_ds(bucket, product, tifs)
primary_name = tifs[0]  #first tif in list
out_prefix_path = 'ws-enduser/USA/conus_mos/' # it gets the "year" folder from the function

xr_write_geotiff_from_ds(DS, primary_name, out_prefix_path)

<xarray.Dataset>
Dimensions:  (x: 31248, y: 12976)
Coordinates:
  * x        (x) float64 -125.0 -125.0 -125.0 -125.0 ... -60.0 -60.0 -60.0
  * y        (y) float64 23.0 23.0 23.0 23.01 23.01 ... 49.99 49.99 50.0 50.0
Data variables:
    etasw    (y, x) float64 -7.846e+39 -7.846e+39 ... -7.846e+39 -7.846e+39
Attributes:
    transform:           (0.0020803384719955188, 0.0, -125.00034453048596, 0....
    crs:                 +init=epsg:4326
    res:                 (0.0020803384719955188, -0.0020810045000000006)
    is_tiled:            1
    nodatavals:          (nan,)
    scales:              (1.0,)
    offsets:             (0.0,)
    descriptions:        ('etasw_',)
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  NEAREST
USA/r37.0_tile0/2008/etasw_200812.tif
ws-enduser/USA/conus_mos/
OUTPUT==ws-enduser/USA/conus_mos/2008/etasw_200812.tif


Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.
Reading input: /home/jovyan/notebooks/etasw_200812.tif
Adding overviews...
Updating dataset tags...
Writing output to: /home/jovyan/notebooks/COG_etasw_200812.tif


a= ['ws-enduser', 'USA', 'conus_mos', '2008', 'etasw_200812.tif']
the BUCKET= ws-enduser
PUSH ws-enduser USA/conus_mos/2008/etasw_200812.tif


In [17]:
! aws s3 ls s3://ws-enduser/USA/conus_mos/2006/

2021-09-01 14:28:31 1981027450 etasw_200601.tif
2021-09-01 14:33:50 2022782795 etasw_200602.tif
2021-09-01 14:39:04 2048049537 etasw_200603.tif
2021-09-01 14:44:18 2044518743 etasw_200604.tif
2021-09-01 14:49:23 2043548204 etasw_200605.tif
2021-09-01 14:54:31 2043448079 etasw_200606.tif
2021-09-01 14:59:39 2042540006 etasw_200607.tif
2021-09-01 15:05:03 2040479396 etasw_200608.tif
2021-09-01 15:32:01 2036992739 etasw_200609.tif
2021-09-01 15:51:48 2039437848 etasw_200610.tif
2021-09-01 16:01:53 2047394516 etasw_200611.tif
2021-09-01 16:26:11 2059328185 etasw_200612.tif


In [18]:
#! rio info s3://ws-enduser/USA/conus_mos/2000/etasw_200001.tif

In [19]:
#! rio info s3://ws-enduser/USA/conus_mos/2000/etasw_200001.tif | python -m json.tool

In [20]:
#! gdalinfo /vsis3/ws-enduser/USA/conus_mos/2000/2000/etasw_200001.tif - not working yet

### open geoTIFF from an amazon data bucket

In [21]:
# s3_file = 's3://ws-enduser/USA/conus_mos/2000/etasw_200001.tif'

In [22]:
# with rasterio.open(s3_file) as src:
#     print(src.profile)

In [23]:
# with rasterio.open(s3_file) as src:
#     print(src.profile)
#     show(src)
#     array=src.read(1)
# print(array)